<a href="https://colab.research.google.com/github/TanushGoel/Machine-Learning-Playground/blob/master/Santander_Transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
pd.options.mode.use_inf_as_na = False
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
!pip3 install catboost

In [0]:
# Upload cleaned files before this - Larger dataset - will take some time
train = pd.read_csv('transactions_train_cleaned_MICE.csv')
target = train.target
train_id = train.ID_code
train.drop("ID_code", axis=1, inplace=True)
train.drop('target', axis=1, inplace=True)
train.head(10)

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
5,11.4763,-2.3182,12.6080,8.6264,10.9621,3.5609,4.5322,15.2255,3.5855,5.9790,...,-6.3068,6.6025,5.2912,0.4403,14.9452,1.0314,-3.6241,9.7670,12.5809,-4.7602
6,11.8091,-0.0832,9.3494,4.2916,11.1355,-8.0198,6.1961,12.0771,-4.3781,7.9232,...,8.7830,6.4521,3.5325,0.1777,18.3314,0.5845,9.1104,9.1143,10.8869,-3.2097
7,13.5580,-7.9881,13.8776,7.5985,8.6543,0.8310,5.6890,22.3262,5.0647,7.1971,...,13.1700,6.5491,3.9906,5.8061,23.1407,-0.3776,4.2178,9.4237,8.6624,3.4806
8,16.1071,2.4426,13.9307,5.6327,8.8014,6.1630,4.4514,10.1854,-3.1882,9.0827,...,1.4298,14.7510,1.6395,1.4181,14.8370,-1.9940,-1.0733,8.1975,19.5114,4.8453
9,12.5088,1.9743,8.8960,5.4508,13.6043,-16.2859,6.0637,16.8410,0.1287,7.9682,...,0.5543,6.3160,1.0371,3.6885,14.8344,0.4467,14.1287,7.9133,16.2375,14.2514


In [0]:
target.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [0]:
test = pd.read_csv('transactions_test_cleaned_MICE.csv')
test_id = test.ID_code
test.drop("ID_code", axis=1, inplace=True)
test.head(10)

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846
5,5.9862,-2.2913,8.6058,7.0685,14.2465,-8.6761,4.2467,14.7632,1.8790,7.2842,...,-2.1115,7.1178,-0.4249,8.8781,14.9438,-2.2151,-6.0233,9.8117,17.1127,10.8240
6,8.4624,-6.1065,7.3603,8.2627,12.0104,-7.2073,4.1670,13.0809,-4.3004,6.3181,...,12.3609,6.8661,4.0971,8.8484,17.5010,0.0295,7.7443,9.1509,18.4736,5.1499
7,17.3035,-2.4212,13.3989,8.3998,11.0777,9.6449,5.9596,17.8477,-4.8068,7.4643,...,4.4676,4.4214,0.9303,1.4994,15.2648,-1.7931,6.5316,10.4855,23.4631,0.7283
8,6.9856,0.8402,13.7161,4.7749,8.6784,-13.7607,4.3386,14.5843,2.5883,7.2215,...,-3.4657,7.8754,2.4698,-0.0362,16.7144,0.1221,-1.4328,9.9207,16.9865,-3.3304
9,10.3811,-6.9348,14.6690,9.0941,11.9058,-10.8018,3.4508,20.2816,-1.4112,6.7401,...,1.8052,11.0723,0.8907,4.7680,15.1425,0.6075,-4.4447,9.5788,15.8146,9.3457


In [0]:
test_id.head()

0    test_0
1    test_1
2    test_2
3    test_3
4    test_4
Name: ID_code, dtype: object

In [0]:
from sklearn.model_selection import KFold
n_split=5
kf = KFold(n_splits=n_split, random_state=42, shuffle=True)
y_valid_pred = 0 * target
y_test_pred = 0
y_valid_pred.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [0]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool
# bootstrap_type='Bayesian', boosting_type='Ordered' (for smaller datasets), rsm=1 SLOW
# bootstrap_type='Bernoulli', boosting_type='Plain'(for larger datasets), rsm=0.1 FAST
final_clf = CatBoostClassifier(iterations=20000, eval_metric='AUC', loss_function='Logloss', bootstrap_type='Bernoulli', boosting_type='Plain', rsm=1, od_type="Iter", max_depth=10, random_state=42)

for idx, (train_index, valid_index) in enumerate(kf.split(train)):
    X_train, X_valid, y_train, y_valid = train.iloc[train_index,:], train.iloc[valid_index,:], target.iloc[train_index], target.iloc[valid_index]
    cat_train = Pool(X_train, y_train)
    cat_valid = Pool(X_valid, y_valid)
    print("\nNEW FOLD :", idx+1)
    fit_model = final_clf.fit(cat_train, eval_set=cat_valid, early_stopping_rounds=350, verbose=50, use_best_model=True, plot=True)
    pred = fit_model.predict_proba(X_valid)[:,1]
    print("AUC :", roc_auc_score(y_valid, pred))
    pred = y_valid_pred.iloc[valid_index]
    y_test_pred += fit_model.predict_proba(test)[:,1]
y_test_pred /= n_split

In [0]:
sample_sub = pd.read_csv('transactions_sample_submission.csv')
sample_sub.head()

In [0]:
sample_sub.target = y_test_pred
submission = sample_sub
submission.head()

In [0]:
# To Download Submission
submission.to_csv('TRANSACTIONS_SUBMISSION.csv', index=False)
from google.colab import files
files.download('TRANSACTIONS_SUBMISSION.csv')